# EDA: Анализ заявок ЖКХ (MDO Appeals)

## Цель анализа
Провести разведочный анализ данных для задачи предсказания времени выполнения заявок ЖКХ.

**ML-задача:** Регрессия - предсказание количества часов до закрытия заявки (`completed_hours`)

## 1. Импорт библиотек и загрузка данных

In [ ]:
# Основные библиотеки
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Дополнительные библиотеки для анализа
from scipy import stats
from datetime import datetime, timedelta

# Настройка визуализации
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

print('Библиотеки загружены успешно!')

In [ ]:
# Загрузка данных
df = pd.read_csv('mdo_appeals_25221.csv')
print(f'Данные загружены: {df.shape[0]} строк, {df.shape[1]} колонок')
print(f'Период данных: 2024 год')

## 2. Базовые характеристики набора данных

In [ ]:
# Первичный осмотр данных
print("Первые 5 строк данных:")
df.head()

In [ ]:
# Информация о типах данных и пропусках
print("Информация о датасете:")
print("="*50)
df.info()

In [ ]:
# Базовые статистики для числовых переменных
print("Описательные статистики числовых переменных:")
print("="*70)
df.describe().round(2)

### 2.1 Анализ пропущенных значений

In [ ]:
# Подсчет и визуализация пропусков
missing_data = pd.DataFrame({
    'Колонка': df.columns,
    'Пропущенные значения': df.isnull().sum(),
    'Процент пропусков': (df.isnull().sum() / len(df) * 100).round(2)
})
missing_data = missing_data.sort_values('Процент пропусков', ascending=False)

print("Анализ пропущенных значений:")
print("="*50)
print(missing_data.to_string(index=False))

In [ ]:
# Визуализация пропусков с помощью seaborn
plt.figure(figsize=(14, 8))

# График 1: Heatmap пропусков
plt.subplot(2, 1, 1)
sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='RdYlBu')
plt.title('Карта пропущенных значений (желтый = пропуск)', fontsize=14)
plt.xlabel('Признаки')
plt.xticks(rotation=45)

# График 2: Процент заполненности
plt.subplot(2, 1, 2)
filled_percent = 100 - missing_data['Процент пропусков']
colors = ['green' if x == 100 else 'orange' if x > 50 else 'red' for x in filled_percent]
plt.barh(missing_data['Колонка'], filled_percent, color=colors)
plt.xlabel('Процент заполненных значений (%)')
plt.title('Заполненность данных по колонкам', fontsize=14)
plt.xlim(0, 100)

# Добавляем значения на график
for i, v in enumerate(filled_percent):
    plt.text(v + 1, i, f'{v:.1f}%', va='center')

plt.tight_layout()
plt.show()

print("\n💡 Вывод: Как и ожидалось, только year_day и completed_hours полностью заполнены.")
print("   Необходимо будет обработать пропуски в остальных признаках перед обучением модели.")

## 3. Анализ целевой переменной (completed_hours)

In [ ]:
# Статистики целевой переменной
print("Детальная статистика целевой переменной completed_hours:")
print("="*60)
print(f"Среднее время выполнения: {df['completed_hours'].mean():.2f} часов")
print(f"Медиана: {df['completed_hours'].median():.2f} часов")
print(f"Стандартное отклонение: {df['completed_hours'].std():.2f} часов")
print(f"Минимум: {df['completed_hours'].min():.2f} часов")
print(f"Максимум: {df['completed_hours'].max():.2f} часов")
print(f"25-й перцентиль: {df['completed_hours'].quantile(0.25):.2f} часов")
print(f"75-й перцентиль: {df['completed_hours'].quantile(0.75):.2f} часов")
print(f"95-й перцентиль: {df['completed_hours'].quantile(0.95):.2f} часов")
print(f"\nКоэффициент асимметрии: {df['completed_hours'].skew():.2f}")
print(f"Коэффициент эксцесса: {df['completed_hours'].kurtosis():.2f}")

In [ ]:
# Визуализация распределения целевой переменной
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Гистограмма
axes[0, 0].hist(df['completed_hours'], bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].axvline(df['completed_hours'].mean(), color='red', linestyle='--', label=f'Среднее: {df["completed_hours"].mean():.1f}')
axes[0, 0].axvline(df['completed_hours'].median(), color='green', linestyle='--', label=f'Медиана: {df["completed_hours"].median():.1f}')
axes[0, 0].set_xlabel('Часы до закрытия заявки')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].set_title('Распределение времени выполнения заявок')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Box plot
axes[0, 1].boxplot(df['completed_hours'].dropna(), vert=True)
axes[0, 1].set_ylabel('Часы')
axes[0, 1].set_title('Box Plot времени выполнения\n(видны выбросы)')
axes[0, 1].grid(True, alpha=0.3)

# QQ-plot для проверки нормальности
stats.probplot(df['completed_hours'].dropna(), dist="norm", plot=axes[1, 0])
axes[1, 0].set_title('QQ-Plot (проверка на нормальность)')
axes[1, 0].grid(True, alpha=0.3)

# Логарифмическое преобразование
log_hours = np.log1p(df['completed_hours'])
axes[1, 1].hist(log_hours, bins=50, edgecolor='black', alpha=0.7, color='orange')
axes[1, 1].set_xlabel('log(1 + часы)')
axes[1, 1].set_ylabel('Частота')
axes[1, 1].set_title('Распределение после логарифмического преобразования')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Вывод: Распределение целевой переменной имеет правую асимметрию (длинный хвост).")
print("   Многие заявки закрываются быстро, но есть значительные выбросы.")
print("   Логарифмическое преобразование может помочь нормализовать распределение для моделирования.")

### 3.1 Анализ выбросов в целевой переменной

In [ ]:
# Определение выбросов методом IQR
Q1 = df['completed_hours'].quantile(0.25)
Q3 = df['completed_hours'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['completed_hours'] < lower_bound) | (df['completed_hours'] > upper_bound)]
print(f"Границы для определения выбросов (метод IQR):")
print(f"Нижняя граница: {lower_bound:.2f} часов")
print(f"Верхняя граница: {upper_bound:.2f} часов")
print(f"\nКоличество выбросов: {len(outliers)} ({len(outliers)/len(df)*100:.2f}% от всех данных)")
print(f"\nПримеры экстремальных значений (топ-10 самых долгих заявок):")
print(df.nlargest(10, 'completed_hours')[['year_day', 'priority', 'completed_hours']])

print("\n💡 Вывод: Присутствуют значительные выбросы. Заявки с временем выполнения > {} часов".format(int(upper_bound)))
print("   требуют отдельного анализа - возможно, это особые категории или ошибки в данных.")

## 4. Анализ временных паттернов

In [ ]:
# Преобразуем day_of_year в реальную дату (2024 год)
df['date'] = pd.to_datetime('2024-01-01') + pd.to_timedelta(df['year_day'] - 1, unit='D')
df['month'] = df['date'].dt.month
df['weekday'] = df['date'].dt.dayofweek
df['week'] = df['date'].dt.isocalendar().week

# Статистики по месяцам
monthly_stats = df.groupby('month')['completed_hours'].agg([
    'count', 'mean', 'median', 'std'
]).round(2)
monthly_stats.columns = ['Количество', 'Среднее', 'Медиана', 'Ст.откл']

print("Статистика по месяцам:")
print(monthly_stats)

In [ ]:
# Интерактивная визуализация временных паттернов с Plotly
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Количество заявок по дням года', 
                    'Среднее время выполнения по месяцам',
                    'Распределение заявок по дням недели',
                    'Среднее время выполнения по дням недели'),
    specs=[[{'type': 'scatter'}, {'type': 'bar'}],
           [{'type': 'bar'}, {'type': 'box'}]]
)

# График 1: Количество заявок по дням
daily_counts = df.groupby('year_day').size()
fig.add_trace(
    go.Scatter(x=daily_counts.index, y=daily_counts.values, 
               mode='lines', name='Заявки',
               line=dict(color='blue', width=1)),
    row=1, col=1
)

# График 2: Среднее время по месяцам
monthly_avg = df.groupby('month')['completed_hours'].mean()
fig.add_trace(
    go.Bar(x=monthly_avg.index, y=monthly_avg.values,
           name='Среднее время',
           marker_color='lightgreen'),
    row=1, col=2
)

# График 3: Распределение по дням недели
weekday_counts = df['weekday'].value_counts().sort_index()
weekday_names = ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']
fig.add_trace(
    go.Bar(x=weekday_names, y=weekday_counts.values,
           name='Количество',
           marker_color='coral'),
    row=2, col=1
)

# График 4: Box plot времени выполнения по дням недели
for i, day in enumerate(['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс']):
    day_data = df[df['weekday'] == i]['completed_hours']
    fig.add_trace(
        go.Box(y=day_data, name=day,
               marker_color='lightblue'),
        row=2, col=2
    )

fig.update_layout(height=800, showlegend=False, 
                  title_text="Временной анализ заявок ЖКХ")
fig.update_xaxes(title_text="День года", row=1, col=1)
fig.update_xaxes(title_text="Месяц", row=1, col=2)
fig.update_xaxes(title_text="День недели", row=2, col=1)
fig.update_xaxes(title_text="День недели", row=2, col=2)
fig.update_yaxes(title_text="Количество", row=1, col=1)
fig.update_yaxes(title_text="Часы", row=1, col=2)
fig.update_yaxes(title_text="Количество", row=2, col=1)
fig.update_yaxes(title_text="Часы", row=2, col=2)

fig.show()

print("\n💡 Выводы по временным паттернам:")
print("   1. Наблюдаются сезонные колебания в количестве заявок")
print("   2. Время выполнения может зависеть от дня недели")
print("   3. Выходные дни могут иметь особые паттерны обработки заявок")

## 5. Анализ категориальных переменных

In [ ]:
# Анализ приоритетов
print("Распределение заявок по приоритетам:")
print("="*40)
priority_stats = df.groupby('priority').agg({
    'completed_hours': ['count', 'mean', 'median', 'std']
}).round(2)
priority_stats.columns = ['Количество', 'Среднее время', 'Медиана', 'Ст.откл']
print(priority_stats)
print(f"\nПропущенные значения в priority: {df['priority'].isna().sum()}")

In [ ]:
# Визуализация влияния приоритета на время выполнения
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# График 1: Распределение приоритетов
priority_counts = df['priority'].value_counts().sort_index()
axes[0].bar(priority_counts.index, priority_counts.values, color=['green', 'yellow', 'orange', 'red'])
axes[0].set_xlabel('Приоритет')
axes[0].set_ylabel('Количество заявок')
axes[0].set_title('Распределение заявок по приоритетам\n(1-низкий, 4-высокий)')
axes[0].grid(True, alpha=0.3)

# График 2: Время выполнения по приоритетам
df_priority = df.dropna(subset=['priority'])
sns.violinplot(data=df_priority, x='priority', y='completed_hours', ax=axes[1])
axes[1].set_xlabel('Приоритет')
axes[1].set_ylabel('Часы до закрытия')
axes[1].set_title('Распределение времени выполнения по приоритетам')
axes[1].set_ylim(0, df_priority['completed_hours'].quantile(0.95))
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Вывод: Видна зависимость времени выполнения от приоритета.")
print("   Заявки с более высоким приоритетом в среднем выполняются быстрее.")
print("   Но разброс значений велик для всех приоритетов.")

In [ ]:
# Анализ топ категорий
print("Анализ категорий заявок:")
print("="*40)

# Считаем непустые категории
category_counts = df['category_id'].value_counts()
print(f"Всего уникальных категорий: {df['category_id'].nunique()}")
print(f"Заявок без категории: {df['category_id'].isna().sum()}")
print(f"\nТоп-10 категорий по количеству заявок:")

top_categories = category_counts.head(10)
for cat_id, count in top_categories.items():
    avg_time = df[df['category_id'] == cat_id]['completed_hours'].mean()
    print(f"  Категория {int(cat_id)}: {count} заявок, среднее время: {avg_time:.1f} часов")

In [ ]:
# Интерактивный анализ топ-20 категорий с Plotly
top20_categories = df['category_id'].value_counts().head(20).index
df_top20 = df[df['category_id'].isin(top20_categories)]

# Создаем сводную таблицу
category_summary = df_top20.groupby('category_id').agg({
    'completed_hours': ['count', 'mean', 'median']
}).round(2)
category_summary.columns = ['Количество', 'Среднее', 'Медиана']
category_summary = category_summary.sort_values('Количество', ascending=False)

# Интерактивная визуализация
fig = px.scatter(category_summary.reset_index(), 
                 x='Количество', 
                 y='Среднее',
                 size='Медиана',
                 hover_data=['category_id', 'Медиана'],
                 title='Топ-20 категорий: количество vs среднее время выполнения',
                 labels={'Среднее': 'Среднее время (часы)', 
                        'Количество': 'Количество заявок'},
                 color='Среднее',
                 color_continuous_scale='RdYlGn_r')

fig.update_layout(height=500)
fig.show()

print("\n💡 Вывод: Разные категории имеют существенно разное среднее время выполнения.")
print("   Это важный признак для модели предсказания.")
print("   Некоторые категории встречаются часто, другие - редко (проблема дисбаланса).")

## 6. Анализ исполнителей и создателей заявок

In [ ]:
# Анализ исполнителей
print("Анализ исполнителей:")
print("="*40)
print(f"Всего уникальных исполнителей: {df['performer_id'].nunique()}")
print(f"Заявок без назначенного исполнителя: {df['performer_id'].isna().sum()}")

# Топ-10 исполнителей по производительности
performer_stats = df.dropna(subset=['performer_id']).groupby('performer_id').agg({
    'completed_hours': ['count', 'mean', 'median']
}).round(2)
performer_stats.columns = ['Количество', 'Среднее время', 'Медиана']
performer_stats = performer_stats[performer_stats['Количество'] >= 10]  # Только с 10+ заявками

print(f"\nТоп-5 самых быстрых исполнителей (мин 10 заявок):")
print(performer_stats.nsmallest(5, 'Среднее время'))

print(f"\nТоп-5 самых медленных исполнителей (мин 10 заявок):")
print(performer_stats.nlargest(5, 'Среднее время'))

In [ ]:
# Визуализация производительности исполнителей
# Берем топ-20 исполнителей по количеству заявок
top_performers = df['performer_id'].value_counts().head(20).index
df_top_perf = df[df['performer_id'].isin(top_performers)]

plt.figure(figsize=(14, 6))

# Box plot для топ исполнителей
performer_data = []
performer_labels = []
for perf_id in top_performers:
    perf_times = df_top_perf[df_top_perf['performer_id'] == perf_id]['completed_hours']
    if len(perf_times) > 0:
        performer_data.append(perf_times)
        performer_labels.append(f'ID_{int(perf_id)}')

plt.boxplot(performer_data, labels=performer_labels)
plt.xticks(rotation=45)
plt.ylabel('Время выполнения (часы)')
plt.xlabel('ID исполнителя')
plt.title('Распределение времени выполнения для топ-20 исполнителей')
plt.ylim(0, df['completed_hours'].quantile(0.9))
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Вывод: Существует значительная вариация в скорости работы исполнителей.")
print("   ID исполнителя может быть важным признаком для модели.")
print("   Некоторые исполнители стабильно быстрые, другие имеют большой разброс.")

## 7. Корреляционный анализ

In [ ]:
# Подготовка данных для корреляционного анализа
# Выбираем только числовые колонки
numeric_cols = ['year_day', 'priority', 'due_date_hours', 'assigned_hours', 'completed_hours']
df_numeric = df[numeric_cols].copy()

# Вычисляем корреляционную матрицу
correlation_matrix = df_numeric.corr()

# Визуализация корреляционной матрицы
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1,
            cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков', fontsize=14)
plt.tight_layout()
plt.show()

# Вывод корреляций с целевой переменной
print("\nКорреляция признаков с целевой переменной (completed_hours):")
print("="*60)
target_corr = correlation_matrix['completed_hours'].sort_values(ascending=False)
for feature, corr in target_corr.items():
    if feature != 'completed_hours':
        print(f"{feature:20s}: {corr:+.3f}")

print("\n💡 Выводы:")
print("   1. due_date_hours имеет положительную корреляцию с временем выполнения")
print("   2. assigned_hours также коррелирует с completed_hours")
print("   3. Приоритет имеет отрицательную корреляцию (высокий приоритет = быстрее выполнение)")

## 8. Анализ взаимосвязей между признаками

In [ ]:
# Анализ связи между категорией и приоритетом
print("Анализ связи между категорией и приоритетом:")
print("="*50)

# Создаем сводную таблицу
df_cat_prior = df.dropna(subset=['category_id', 'priority'])
top10_cats = df_cat_prior['category_id'].value_counts().head(10).index

priority_by_category = pd.crosstab(
    df_cat_prior[df_cat_prior['category_id'].isin(top10_cats)]['category_id'],
    df_cat_prior[df_cat_prior['category_id'].isin(top10_cats)]['priority'],
    normalize='index'
) * 100

# Визуализация
plt.figure(figsize=(12, 6))
sns.heatmap(priority_by_category, annot=True, fmt='.1f', cmap='YlOrRd',
            cbar_kws={'label': 'Процент заявок'})
plt.title('Распределение приоритетов по топ-10 категориям (%)', fontsize=14)
plt.xlabel('Приоритет')
plt.ylabel('ID категории')
plt.tight_layout()
plt.show()

print("\n💡 Вывод: Некоторые категории чаще имеют определенные приоритеты.")
print("   Но связь не является жесткой - в каждой категории встречаются разные приоритеты.")

In [ ]:
# Анализ времени назначения исполнителя vs времени выполнения
df_assigned = df.dropna(subset=['assigned_hours', 'completed_hours'])

# Scatter plot с регрессионной линией
plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.scatter(df_assigned['assigned_hours'], df_assigned['completed_hours'], 
            alpha=0.3, s=10)
plt.xlabel('Время до назначения исполнителя (часы)')
plt.ylabel('Время до закрытия заявки (часы)')
plt.title('Зависимость времени выполнения от времени назначения')

# Добавляем линию y=x для сравнения
max_val = min(df_assigned[['assigned_hours', 'completed_hours']].quantile(0.95).max(), 500)
plt.plot([0, max_val], [0, max_val], 'r--', alpha=0.5, label='y=x')
plt.xlim(0, max_val)
plt.ylim(0, max_val)
plt.legend()
plt.grid(True, alpha=0.3)

# График 2: Время работы после назначения
plt.subplot(1, 2, 2)
work_time = df_assigned['completed_hours'] - df_assigned['assigned_hours']
work_time = work_time[work_time >= 0]  # Убираем отрицательные значения (ошибки данных)

plt.hist(work_time, bins=50, edgecolor='black', alpha=0.7)
plt.axvline(work_time.mean(), color='red', linestyle='--', 
            label=f'Среднее: {work_time.mean():.1f} ч')
plt.axvline(work_time.median(), color='green', linestyle='--',
            label=f'Медиана: {work_time.median():.1f} ч')
plt.xlabel('Время работы после назначения (часы)')
plt.ylabel('Количество заявок')
plt.title('Распределение фактического времени работы')
plt.xlim(0, work_time.quantile(0.95))
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nСтатистика времени работы после назначения:")
print(f"Среднее: {work_time.mean():.2f} часов")
print(f"Медиана: {work_time.median():.2f} часов")
print(f"75-й перцентиль: {work_time.quantile(0.75):.2f} часов")

print("\n💡 Вывод: Время назначения исполнителя сильно влияет на общее время закрытия.")
print("   Быстрое назначение критично для общей эффективности.")

## 9. Выявление аномалий и проблемных областей

In [ ]:
# Поиск аномалий
print("Анализ потенциальных аномалий и проблем в данных:")
print("="*60)

# 1. Заявки с отрицательным временем работы
df_temp = df.dropna(subset=['assigned_hours', 'completed_hours'])
anomaly_negative = df_temp[df_temp['completed_hours'] < df_temp['assigned_hours']]
print(f"1. Заявки, закрытые ДО назначения исполнителя: {len(anomaly_negative)}")
if len(anomaly_negative) > 0:
    print(f"   Это {len(anomaly_negative)/len(df_temp)*100:.2f}% от заявок с обеими датами")

# 2. Сверхбыстрые заявки
super_fast = df[df['completed_hours'] < 0.5]  # Менее 30 минут
print(f"\n2. Сверхбыстрые заявки (<30 минут): {len(super_fast)}")

# 3. Сверхдолгие заявки
super_slow = df[df['completed_hours'] > 30*24]  # Более 30 дней
print(f"\n3. Сверхдолгие заявки (>30 дней): {len(super_slow)}")
if len(super_slow) > 0:
    print("   Примеры:")
    print(super_slow[['year_day', 'priority', 'category_id', 'completed_hours']].head())

# 4. Заявки без исполнителя но закрытые
no_performer = df[df['performer_id'].isna() & df['completed_hours'].notna()]
print(f"\n4. Закрытые заявки без назначенного исполнителя: {len(no_performer)}")

print("\n💡 Рекомендации по обработке аномалий:")
print("   - Проверить корректность данных для аномальных записей")
print("   - Рассмотреть возможность фильтрации экстремальных выбросов")
print("   - Создать отдельные модели для разных типов заявок")

## 10. Итоговые выводы и рекомендации для моделирования

In [ ]:
print("="*70)
print("ИТОГОВЫЕ ВЫВОДЫ ПО EDA")
print("="*70)

print("\n📊 КЛЮЧЕВЫЕ НАХОДКИ:")
print("\n1. ЦЕЛЕВАЯ ПЕРЕМЕННАЯ:")
print("   • Сильная правосторонняя асимметрия (много быстрых заявок, есть долгие)")
print("   • Присутствуют значительные выбросы (>1000 часов)")
print("   • Рекомендуется логарифмическое преобразование или robust методы")

print("\n2. ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ:")
print("   • Значительное количество пропусков во всех признаках кроме year_day и completed_hours")
print("   • Необходима стратегия импутации (median/mode) или использование моделей, устойчивых к пропускам")

print("\n3. ВАЖНЫЕ ПРИЗНАКИ:")
print("   • priority - отрицательная корреляция с временем выполнения")
print("   • assigned_hours и due_date_hours - положительная корреляция")
print("   • category_id - разные категории имеют разное среднее время")
print("   • performer_id - существенные различия между исполнителями")

print("\n4. ВРЕМЕННЫЕ ПАТТЕРНЫ:")
print("   • Есть сезонность и зависимость от дня недели")
print("   • Рекомендуется добавить признаки: месяц, день недели, квартал")

print("\n🔧 РЕКОМЕНДАЦИИ ДЛЯ ПРЕДОБРАБОТКИ:")
print("\n1. Обработка выбросов:")
print("   • Рассмотреть удаление заявок >95 перцентиля или применение winsorization")
print("   • Отдельно проанализировать сверхдолгие заявки (возможно, это особый тип)")

print("\n2. Feature Engineering:")
print("   • Создать признак 'время_до_назначения' = assigned_hours")
print("   • Добавить статистики по исполнителю (средняя скорость работы)")
print("   • Добавить статистики по категории (средняя сложность)")
print("   • Временные признаки: день недели, выходной/рабочий, месяц, квартал")

print("\n3. Обработка категориальных признаков:")
print("   • Target encoding для category_id и performer_id (много уникальных значений)")
print("   • One-hot encoding для priority (мало уникальных значений)")

print("\n4. Валидация:")
print("   • Использовать временную валидацию (train на более ранних, test на поздних)")
print("   • Учесть дисбаланс категорий при разбиении")

print("\n🎯 ВЫБОР МОДЕЛИ:")
print("   • Начать с простой линейной регрессии (baseline)")
print("   • Random Forest / XGBoost для работы с пропусками и нелинейностями")
print("   • Рассмотреть квантильную регрессию для предсказания интервалов")

print("\n" + "="*70)
print("Анализ завершен. Данные готовы для построения модели!")
print("="*70)